In [33]:
from ccnet_spark.pipe_save import load_all,convert_to_absolute_path
import time
from pyspark.sql import SparkSession
import sys
import subprocess
import os
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [2]:
spark = (
    SparkSession.builder.appName("ccnetspark_profile_profile")
    .master("local[*]")
    .config("spark.executor.memory", "40g")
    .config("spark.driver.memory", "40g")
    .config("spark.dynamicAllocation.enabled", "false")
    .config("spark.driver.maxResultSize", "40g")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    # .config("spark.sql.autoBroadcastJoinThreshold","-1")
    # .config("spark.executor.extraJavaOptions", "-XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps")
    # .config("spark.driver.extraJavaOptions", "-XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps")
    .getOrCreate()
)

In [67]:
def get_result(path_name):
    isSample=True
    min_len=300
    sampleRate=1
    dump="2019-18"
    save_type="file"
    output_dir="/metadata0/wxl_data/cached_data/"
    saved_sdf_name = "_sampleRate_" + str(int(sampleRate*100 if isSample else 100)) + "_min_len_"+str(min_len)+".parquet"
    saved_sdf_path = os.path.join(output_dir,path_name,dump,saved_sdf_name)
    saved_sdf_path = convert_to_absolute_path(saved_sdf_path)
    df = spark.read.parquet(f"{save_type}:///{saved_sdf_path}")
    return df
def analy_df(df):
    # 定义聚合函数求和
    sum_columns = [
        F.sum('original_length').alias('sum_original_length'),
        F.sum('length').alias('sum_length'),
        F.sum('nlines').alias('sum_nlines'),
        F.sum('original_nlines').alias('sum_original_nlines')
    ]
    # 按照 'bucket' 和 'lang' 字段进行分组，并计算每个组合的数量和求和
    aggregated_df = df.groupBy('bucket', 'lang').agg(
        F.count('*').alias('count'), *sum_columns
    )
    # 添加两列做比较
    aggregated_df = aggregated_df.withColumn('length_ratio', F.col('sum_length') / F.col('sum_original_length'))
    aggregated_df = aggregated_df.withColumn('nlines_ratio', F.col('sum_nlines') / F.col('sum_original_nlines'))
    aggregated_df = aggregated_df.withColumn("n_count", col("count").cast(IntegerType()))
    # 显示聚合后的结果
    # aggregated_df.show(50)
    return aggregated_df

In [68]:
def show_cal(df):
    lang_counts = df.groupBy("lang").agg(sum("count").alias("total_count"))
    lang_counts.show()
    bucket_counts=df.groupBy("bucket").agg(sum("count").alias("total_count"))
    bucket_counts.show()

In [71]:
df_nokeep=get_result("result_sdf_parquet_nokeep")
df_nokeep_an=analy_df(df_nokeep)


24/04/25 08:50:14 WARN DataSource: All paths were ignored:
  file:/metadata0/wxl_data/cached_data/result_sdf_parquet_nokeep/2019-18/_sampleRate_100_min_len_300.parquet


In [76]:
df_nokeep_an.agg(sum("n_count")).show()

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [69]:
df_nokeep_an.agg(sum("count").alias("count_sum"))

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [64]:
df_nokeep_an

DataFrame[bucket: string, lang: string, count: int, sum_original_length: bigint, sum_length: bigint, sum_nlines: bigint, sum_original_nlines: bigint, length_ratio: double, nlines_ratio: double]

In [62]:

df_nokeep_an.agg(sum("count"))

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [51]:
df_nokeep_an.groupBy("lang").agg({"lang": sum("count")})

TypeError: unsupported operand type(s) for +: 'int' and 'str'

24/04/25 08:39:28 WARN DataSource: All paths were ignored:
  file:/metadata0/wxl_data/cached_data/result_sdf_parquet_nokeep/2019-18/_sampleRate_100_min_len_300.parquet
24/04/25 08:39:29 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [24]:
df_nokeep_an.show()

+------+----+------+-------------------+----------+----------+-------------------+-------------------+-------------------+
|bucket|lang| count|sum_original_length|sum_length|sum_nlines|sum_original_nlines|       length_ratio|       nlines_ratio|
+------+----+------+-------------------+----------+----------+-------------------+-------------------+-------------------+
|  tail|  en|393731|         2525061403| 980046025|  10406761|           68352621| 0.3881276011092709|0.15225108924499034|
|middle|  en|120537|          971723594| 531501860|   3198078|           18023596| 0.5469681535796896| 0.1774383979756315|
|   all|null|149888|         1112207560| 335905864|   3905003|           33186661|0.30201724577380146|0.11766784853709748|
|  head|  en| 88876|          712600213| 404164082|   1819529|           12204283| 0.5671680622975059|0.14908938116233456|
|  tail|  ru| 81920|          547078052| 179232477|   2029848|           16949451|0.32761774365607343|0.11975892316512199|
|  tail|  zh| 49

[Row(lang='en', count(lang)=603144),
 Row(lang=None, count(lang)=0),
 Row(lang='ru', count(lang)=128466),
 Row(lang='zh', count(lang)=80653),
 Row(lang='fr', count(lang)=73058),
 Row(lang='de', count(lang)=88570),
 Row(lang='ja', count(lang)=77437),
 Row(lang='es', count(lang)=64655),
 Row(lang='ar', count(lang)=8476),
 Row(lang='he', count(lang)=3295),
 Row(lang='it', count(lang)=36390),
 Row(lang='tr', count(lang)=12238),
 Row(lang='nl', count(lang)=27445),
 Row(lang='lt', count(lang)=2020),
 Row(lang='pt', count(lang)=28883),
 Row(lang='pl', count(lang)=24728),
 Row(lang='uk', count(lang)=6177),
 Row(lang='vi', count(lang)=10665),
 Row(lang='ro', count(lang)=6030),
 Row(lang='cs', count(lang)=15573),
 Row(lang='ko', count(lang)=8165),
 Row(lang='hu', count(lang)=7584),
 Row(lang='sk', count(lang)=3571),
 Row(lang='sv', count(lang)=10812),
 Row(lang='fa', count(lang)=10617),
 Row(lang='th', count(lang)=4568),
 Row(lang='sl', count(lang)=1298),
 Row(lang='el', count(lang)=7685),
 Row(

In [12]:
df_nokeep.select("lang").distinct().show()

+----+
|lang|
+----+
|  en|
|null|
|  ru|
|  zh|
|  fr|
|  de|
|  ja|
|  es|
|  ar|
|  he|
|  it|
|  tr|
|  nl|
|  lt|
|  pt|
|  pl|
|  uk|
|  vi|
|  ro|
|  cs|
+----+
only showing top 20 rows



In [18]:
df_nokeep.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+---------------+--------------------+------+--------------------+--------------------+------+-----+----------+----+------+
|              digest|                 url|       date_download|       source_domain|          cc_segment|original_length|original_nlines|               title|nlines|         raw_content|         raw_line_id|length|score|perplexity|lang|bucket|
+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+---------------+--------------------+------+--------------------+--------------------+------+-----+----------+----+------+
|sha1:227OGOO5YHD5...|http://www.istudi...|2019-04-18T11:12:51Z|     www.istudio.com|CC-MAIN-201904181...|           1080|             34|iStudio - Darren ...|     7|Join: 13 Jun 2014...|[10, 11, 14, 17, ...|   751| 0.92|     531.4|  en|  tail|
|sha1:22LABVMGWGOY..

In [19]:
df_nokeep.count()
df_nokeep.filter(col("lang") == "null").count()

0

In [15]:
# 首先对 "lang" 列进行去重
distinct_lang_df = df_nokeep.select("lang").distinct()

# 对每种语言进行计数
lang_counts = df_nokeep.groupBy("lang").agg({"lang": "count"})


In [17]:
lang_counts.collect()

[Row(lang='en', count(lang)=603144),
 Row(lang=None, count(lang)=0),
 Row(lang='ru', count(lang)=128466),
 Row(lang='zh', count(lang)=80653),
 Row(lang='fr', count(lang)=73058),
 Row(lang='de', count(lang)=88570),
 Row(lang='ja', count(lang)=77437),
 Row(lang='es', count(lang)=64655),
 Row(lang='ar', count(lang)=8476),
 Row(lang='he', count(lang)=3295),
 Row(lang='it', count(lang)=36390),
 Row(lang='tr', count(lang)=12238),
 Row(lang='nl', count(lang)=27445),
 Row(lang='lt', count(lang)=2020),
 Row(lang='pt', count(lang)=28883),
 Row(lang='pl', count(lang)=24728),
 Row(lang='uk', count(lang)=6177),
 Row(lang='vi', count(lang)=10665),
 Row(lang='ro', count(lang)=6030),
 Row(lang='cs', count(lang)=15573),
 Row(lang='ko', count(lang)=8165),
 Row(lang='hu', count(lang)=7584),
 Row(lang='sk', count(lang)=3571),
 Row(lang='sv', count(lang)=10812),
 Row(lang='fa', count(lang)=10617),
 Row(lang='th', count(lang)=4568),
 Row(lang='sl', count(lang)=1298),
 Row(lang='el', count(lang)=7685),
 Row(

In [7]:
an_df.show()

+------+----+------+-------------------+----------+----------+-------------------+-------------------+-------------------+
|bucket|lang| count|sum_original_length|sum_length|sum_nlines|sum_original_nlines|       length_ratio|       nlines_ratio|
+------+----+------+-------------------+----------+----------+-------------------+-------------------+-------------------+
|  tail|  en|393731|         2525061403| 980046025|  10406761|           68352621| 0.3881276011092709|0.15225108924499034|
|middle|  en|120537|          971723594| 531501860|   3198078|           18023596| 0.5469681535796896| 0.1774383979756315|
|   all|null|149888|         1112207560| 335905864|   3905003|           33186661|0.30201724577380146|0.11766784853709748|
|  head|  en| 88876|          712600213| 404164082|   1819529|           12204283| 0.5671680622975059|0.14908938116233456|
|  tail|  ru| 81920|          547078052| 179232477|   2029848|           16949451|0.32761774365607343|0.11975892316512199|
|  tail|  zh| 49

In [8]:
an_df.sum('count')

AttributeError: 'DataFrame' object has no attribute 'sum'

24/04/24 17:17:20 WARN DataSource: All paths were ignored:
  file:/metadata0/wxl_data/cached_data/result_sdf_parquet_nokeep/2019-18/_sampleRate_100_min_len_300.parquet


+------+----+------+-------------------+----------+----------+-------------------+-------------------+-------------------+
|bucket|lang| count|sum_original_length|sum_length|sum_nlines|sum_original_nlines|       length_ratio|       nlines_ratio|
+------+----+------+-------------------+----------+----------+-------------------+-------------------+-------------------+
|  tail|  en|393731|         2525061403| 980046025|  10406761|           68352621| 0.3881276011092709|0.15225108924499034|
|middle|  en|120537|          971723594| 531501860|   3198078|           18023596| 0.5469681535796896| 0.1774383979756315|
|   all|null|149888|         1112207560| 335905864|   3905003|           33186661|0.30201724577380146|0.11766784853709748|
|  head|  en| 88876|          712600213| 404164082|   1819529|           12204283| 0.5671680622975059|0.14908938116233456|
|  tail|  ru| 81920|          547078052| 179232477|   2029848|           16949451|0.32761774365607343|0.11975892316512199|
|  tail|  zh| 49

In [23]:
aggregated_df

DataFrame[digest: string, url: string, date_download: string, source_domain: string, cc_segment: string, original_length: int, original_nlines: bigint, title: string, nlines: bigint, raw_content: string, raw_line_id: array<int>, length: int, score: float, perplexity: float, lang: string, bucket: string]

In [24]:
df_keep=get_result("result_sdf_parquet_keep_dup")
analy_df(df_keep)

24/04/24 17:17:30 WARN DataSource: All paths were ignored:
  file:/metadata0/wxl_data/cached_data/result_sdf_parquet_keep_dup/2019-18/_sampleRate_100_min_len_300.parquet


+------+----+------+-------------------+----------+----------+-------------------+-------------------+-------------------+
|bucket|lang| count|sum_original_length|sum_length|sum_nlines|sum_original_nlines|       length_ratio|       nlines_ratio|
+------+----+------+-------------------+----------+----------+-------------------+-------------------+-------------------+
|  tail|  en|444027|         2773354838|1279649452|  15639698|           76226491| 0.4614084842179146|0.20517405162989857|
|   all|null|144335|         1125469806| 448723394|   5910575|           32780403|0.39869874039073067|0.18030818596098408|
|middle|  en|113948|          951540264| 583207972|   3772790|           16705055| 0.6129093997014508| 0.2258472061301205|
|  tail|  ru| 91321|          595805905| 243268990|   3366374|           18751140| 0.4083024151967745|0.17952903130156353|
|  head|  en| 72827|          616805975| 387317929|   1838415|            9796985| 0.6279412727803099|0.18765109878192118|
|  tail|  zh| 54

In [17]:
aggregated_df.filter(aggregated_df['lang'] == 'en').orderBy('bucket').show()

+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+---------------+--------------------+------+--------------------+--------------------+------+-----+----------+----+------+
|              digest|                 url|       date_download|       source_domain|          cc_segment|original_length|original_nlines|               title|nlines|         raw_content|         raw_line_id|length|score|perplexity|lang|bucket|
+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+---------------+--------------------+------+--------------------+--------------------+------+-----+----------+----+------+
|sha1:22FDQAZTHZFG...|https://www.proz....|2019-04-18T11:06:20Z|        www.proz.com|CC-MAIN-201904181...|           9207|            404|Working magic in ...|     8|Translation - art...|[254, 258, 259, 2...|  1193|  0.9|     314.7|  en|  head|
|sha1:225F4EG7W6SB..

In [8]:
import matplotlib.pyplot as plt
aggregated_df=df_nokeep
# 从聚合后的DataFrame中提取每种语言的数据
lang_data = {
    'en': aggregated_df.filter(aggregated_df['lang'] == 'en').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'ru': aggregated_df.filter(aggregated_df['lang'] == 'ru').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'zh': aggregated_df.filter(aggregated_df['lang'] == 'zh').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'fr': aggregated_df.filter(aggregated_df['lang'] == 'fr').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'de': aggregated_df.filter(aggregated_df['lang'] == 'de').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'ja': aggregated_df.filter(aggregated_df['lang'] == 'ja').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'es': aggregated_df.filter(aggregated_df['lang'] == 'es').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'ar': aggregated_df.filter(aggregated_df['lang'] == 'ar').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'he': aggregated_df.filter(aggregated_df['lang'] == 'he').orderBy('bucket').select('bucket', 'length_ratio').collect(),
    'it': aggregated_df.filter(aggregated_df['lang'] == 'it').orderBy('bucket').select('bucket', 'length_ratio').collect()
}

# 绘制折线图
plt.figure(figsize=(10, 6))

for lang, data in lang_data.items():
    buckets = [row['bucket'] for row in data]
    length_ratios = [row['length_ratio'] for row in data]
    plt.plot(buckets, length_ratios, label=lang)

plt.xlabel('Bucket')
plt.ylabel('Length Ratio')
plt.title('Length Ratio by Language')
plt.legend()
plt.grid(True)
plt.show()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `length_ratio` cannot be resolved. Did you mean one of the following? [`length`, `lang`, `digest`, `perplexity`, `title`].;
'Project [bucket#15, 'length_ratio]
+- Sort [bucket#15 ASC NULLS FIRST], true
   +- Filter (lang#14 = en)
      +- Relation [digest#0,url#1,date_download#2,source_domain#3,cc_segment#4,original_length#5,original_nlines#6L,title#7,nlines#8L,raw_content#9,raw_line_id#10,length#11,score#12,perplexity#13,lang#14,bucket#15] parquet
